In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

with open("data/input.txt", "r") as text:
    text = text.read()

# get unique chars in training data
chars = sorted(list(set(text)))
vocab_size = len(chars)

# map string to integers (idx)
stoi = { ch:i for i,ch in enumerate(chars) } # string to int
itos = { i:ch for i,ch in enumerate(chars) } # int to string

# char encode/decode
encode = lambda s: [stoi[c] for c in s] # string -> list of ints
decode = lambda l: ''.join([itos[i] for i in l]) # list of ints -> string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_length, (batch_size,))
    x = torch.stack([data[i:i+context_length] for i in ix])
    y = torch.stack([data[i+1:i+context_length+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# hyperparameters
batch_size = 64
context_length = 256
max_iters = 5000
eval_interval = 250
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200


In [14]:
from model import LanguageModel

model = LanguageModel()
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
model.to(device)
if torch.cuda.device_count() > 1:
    model = model.module

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

losses = []

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        loss = estimate_loss(model)
        losses.append(loss)
        print(f"step {iter}: train loss {loss['train']:.4f}, val loss {loss['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


10.788929 M parameters
step 0: train loss 4.2849, val loss 4.2823
step 250: train loss 2.3792, val loss 2.4047
step 500: train loss 2.0025, val loss 2.0908
step 750: train loss 1.7495, val loss 1.8903
step 1000: train loss 1.6016, val loss 1.7750
step 1250: train loss 1.5095, val loss 1.6967
step 1500: train loss 1.4391, val loss 1.6454
step 1750: train loss 1.3884, val loss 1.6061
step 2000: train loss 1.3467, val loss 1.5652
step 2250: train loss 1.3079, val loss 1.5495
step 2500: train loss 1.2771, val loss 1.5272
step 2750: train loss 1.2530, val loss 1.5144
step 3000: train loss 1.2293, val loss 1.5065
step 3250: train loss 1.2099, val loss 1.5059
step 3500: train loss 1.1886, val loss 1.4904
step 3750: train loss 1.1674, val loss 1.4805
step 4000: train loss 1.1483, val loss 1.4824
step 4250: train loss 1.1325, val loss 1.4864
step 4500: train loss 1.1102, val loss 1.4831
step 4750: train loss 1.0932, val loss 1.4854
step 4999: train loss 1.0760, val loss 1.4827


In [ ]:
torch.save(model.state_dict(), 'model.pth')